In [1]:
import requests
from bs4 import BeautifulSoup
from tqdm import tqdm
import numpy as np
import pandas as pd
import re
from datetime import datetime
import time
from selenium import webdriver
import math
import matplotlib.pyplot as plt
from unidecode import unidecode
from sklearn.cluster import KMeans
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
import seaborn as sns

## Loading data

In [2]:
data = pd.read_csv('devices.csv')

## PCA

**dropping NaNs for quick PCA**

Now, the column with most missings has 6% of missing values, so it might be ok do drop those...

In [3]:
print('numero de aparelhos antes do drop: ', data.shape[0]_
data = data.dropna()
print('numero de aparelhos depois do drop: ', data.shape[0])

SyntaxError: invalid syntax (<ipython-input-3-0d92d259d9be>, line 2)

In [ ]:
X = data.drop(columns=['modelo_celular'])
X = pd.get_dummies(X, prefix='prop_')

We will create functions that do things:
- Apply PCA tecnique to help us see all columns as two variables. This will improve analysis and see how people are distanced...

In [ ]:
# def pca_fit():
X = X
n_components = 2
# instantiate and fit the pca object
X_std = StandardScaler().fit_transform(X)
# X_std = X

pca = PCA(n_components)
pca.fit(X_std)

print(f'each pca explains: {pca.explained_variance_ratio_}')
explains = pca.explained_variance_ratio_.sum()
print(f'Both components explain together {round(explains*100,2)}% of the data')

#now transform!
X_pca = pca.transform(X_std)

#check results
print("\noriginal shape:   ", X.shape)
print("\ntransformed shape:", X_pca.shape)
pc1 = X_pca[:,0]
pc2 = X_pca[:,1]

In [ ]:
c = data.modelo_celular.apply(lambda x: 'blue' if 'iphone' in x \
                                          else ('red' if 'samsung' in x \
                                                else ('green' if 'huawei' in x \
                                                      else ('black' if 'motorola' in x else 'yellow'))))

In [ ]:
# c = np.array(data.ios.apply(lambda x: 'blue' if x != 0 else 'red'))
# c = data.modelo_celular.apply(lambda x: 'blue' if 'iphone' in x else ('red' if 'samsung' in x else 'yellow'))
plt.scatter(x=pc1, y=pc2, c=c)

- We will also create a function for ploting Kmeans using different number of clusters

## KMeas: dividing all smartphones in groups of 4-5

In [ ]:
#making clusters of somewhat 5 smartphones
n_clusters = round(X.shape[0]/5)

kmeans = KMeans(n_clusters)

kmeans.fit(X_std)

labels = kmeans.labels_

#add to the dataframe as groups
data['cluster'] = labels + 1

#generate data for plot
pc1 = X_pca[:,0]
pc2 = X_pca[:,1]

c_labels = kmeans.labels_

- Now let us see the results

In [ ]:
fig, axis = plt.subplots(nrows=1,ncols=1,figsize=(6,6))

axis.scatter(x=pc1, y=pc2, c=c_labels, cmap='rainbow')
axis.set_title(f'clusters: {n_clusters}')
axis.set_xlabel('pca-1')
axis.set_ylabel('pca-2')
plt.show()